In [ ]:
import pandas as pd
from darts.models import DLinearModel, RegressionModel
from darts import TimeSeries
import pickle


inflation_df = pd.read_csv("Inflation.csv", index_col=0, header = [0,1])
CPI_df = pd.read_csv("CPI.csv", index_col=0, header = [0,1])

inflation_df.columns = inflation_df.columns.droplevel(1)

inflation_df.index = pd.to_datetime(inflation_df.index.astype(str), format='%Y%m')

inflation_df = inflation_df.iloc[2:]

inflation_df_train = inflation_df[inflation_df.index < pd.Timestamp('2000-03-01')]
inflation_df_test = inflation_df[inflation_df.index >= pd.Timestamp('2000-03-01')]

inflation_series = TimeSeries.from_dataframe(inflation_df)

country_names = inflation_df.columns[:-17]

model = DLinearModel(

    input_chunk_length=4,

    output_chunk_length=1,

    kernel_size=12,

    batch_size = 20,

    n_epochs=20,

    pl_trainer_kwargs={
                    "accelerator": "gpu",
                    "devices": [0],
                    "enable_progress_bar": False
                      }
)



## All countries 1 by 1

In [5]:
#forecasts_country = {}
#for country_name in country_names:
#  print(country_name)
#  forecasts = model.historical_forecasts(
#                                                  inflation_series[country_name],
#                                                  past_covariates=inflation_series.drop_columns(country_name),
#                                                  start=pd.Timestamp('2000-03-01'),  # Start forecasting from this date
#                                                  forecast_horizon=1,  # Predict 1 step ahead at each iteration
#                                                  stride=1,  # Move forward 1 step at a time
#                                                  retrain=True,  # Retrain the model at each step
#                                                  train_length=240,  # Fixed rolling window size train_size
#                                                  verbose=True,
#                                                  enable_optimization=False
#                                                  )
#  forecasts_country[country_name] = forecasts
#
#with open('forecast_dict.pkl', 'wb') as f:
#    pickle.dump(forecasts, f)

## All countries at once

In [ ]:
all_forecasts = model.historical_forecasts(
                                                inflation_series[list(country_names)],
                                                past_covariates=inflation_series.drop_columns(list(country_names)),
                                                start=pd.Timestamp('2000-03-01'),  # Start forecasting from this date
                                                forecast_horizon=1,  # Predict 1 step ahead at each iteration
                                                stride=1,  # Move forward 1 step at a time
                                                retrain=True,  # Retrain the model at each step
                                                train_length=240,  # Fixed rolling window size train_size
                                                verbose=True,
                                                enable_optimization=False)

with open('all_forecast.pkl', 'wb') as f:
    pickle.dump(all_forecasts, f)

In [8]:
import pandas as pd

all_forecasts = pd.read_pickle(r'all_forecast.pkl')

In [9]:
from darts.metrics.metrics import rmse, mae
import numpy as np

np.mean([rmse(inflation_series.drop_before(pd.Timestamp("2000-02-01"))[country_name], all_forecasts[country_name]) for country_name in country_names])


0.829759792161182

In [27]:
np.mean([mae(inflation_series.drop_before(pd.Timestamp("2000-02-01"))[country_name], all_forecasts[country_name]) for country_name in country_names])

0.5727243126195439